In [ ]:
from datetime import datetime
import pandas as pd
import plotly.express as px

from app.data.utils.bigquery import BigQuery

bq = BigQuery()

In [ ]:
params = {
    'start_date': '2022-11-08',
    'end_date': '2022-11-17',
    'site': 'MLA',
    'click_window': 24 * 60 * 60
}

sql = '''
WITH prints AS (
    SELECT
        `id`,
        `ds`,
        `event_data`,
         json_extract_scalar(`event_data`, '$.content_source') AS content_source,
         json_extract_scalar(`event_data`, '$.print_id') AS print_id,
        `user`.`user_id` AS `user_id`,
        CAST(REPLACE(`server_timestamp`, "T", ' ') AS TIMESTAMP) AS ts
    FROM
        meli-bi-data.MELIDATA.ADVERTISING
    WHERE
        `ds` >= '{start_date}'
        AND `ds` < '{end_date}'
        AND `site` = '{site}'
        AND `event` = 'display_prints'
        AND (NOT
            REGEXP_CONTAINS(LOWER(`device`.user_agent),
            '.*(libwww|wget|lwp|damnBot|bbbike|java|spider|crawl|slurp|bot|feedburner|googleimageproxy|google web preview).*')
        )
),
clicks AS (
    SELECT
        `ds`,
        `event`,
        `user`.`user_id` AS `user_id`,
        json_extract_scalar(`event_data`, '$.print_id') AS print_id,
        json_extract_scalar(`event_data`, '$.click_id') AS click_id,
        CAST(REPLACE(`server_timestamp`, "T", ' ') AS TIMESTAMP) AS ts
    FROM
        meli-bi-data.MELIDATA.ADVERTISING
    WHERE
        `ds` >= '{start_date}'
        AND `ds` < "{end_date}"
        AND `site` = '{site}'
        AND `event` = 'display_clicks'
        AND (NOT
            REGEXP_CONTAINS(LOWER(`device`.user_agent),
            '.*(libwww|wget|lwp|damnBot|bbbike|java|spider|crawl|slurp|bot|feedburner|googleimageproxy|google web preview).*')
        )
),
prints_clicks AS (
    SELECT
        prints.id AS id,
        prints.ds AS `cday`,
        EXTRACT(HOUR FROM prints.`ts`) AS `chour`,
        prints.content_source,
        CAST(
            json_extract_scalar(prints.event_data, '$.campaign_id') AS INT
        ) AS campaign_id,
        CAST(json_extract_scalar(prints.event_data, '$.line_item_id') AS INT64) AS line_item_id,
        CAST(json_extract_scalar(prints.event_data, '$.creative_id') AS INT64) AS creative_id,
        CASE
            WHEN clicks.ds IS NOT NULL THEN 1 
            ELSE 0
        END AS target
    FROM prints
    LEFT JOIN clicks
    ON prints.print_id = clicks.print_id
        AND ((prints.user_id = clicks.user_id) OR clicks.user_id IS NULL)
        AND TIMESTAMP_DIFF(clicks.ts, prints.ts, SECOND) <= {click_window}
),
grouped AS
(
    SELECT
        cday,
        chour,
        content_source,
        campaign_id,
        line_item_id,
        creative_id,
        SUM(target) AS n_clicks,
        COUNT(*) AS n_prints
    FROM prints_clicks
    GROUP BY 1,2,3,4,5,6
)/*,
artifact_grouped AS
(
    SELECT
        cday,
        content_source,
        campaign_id,
        line_item_id,
        creative_id,
        SUM(n_clicks) AS n_clicks,
        sum(n_prints) AS n_prints
    FROM meli-bi-data.SBOX_DSPCREATIVOS.CLICKS_PRINTS_PER_HOUR
    GROUP BY 1,2,3,4,5
)*/

SELECT a.*, b.n_clicks AS artifact_n_clicks, b.n_prints AS artifact_n_prints
FROM grouped a
FULL JOIN meli-bi-data.SBOX_DSPCREATIVOS.CLICKS_PRINTS_PER_HOUR b
USING(cday, chour, content_source, campaign_id, line_item_id, creative_id)
'''

sql = sql.format(**params)

In [ ]:
df = bq.run_query(sql)
df = df[(~pd.isnull(df)).apply(all, axis = 1)]

In [ ]:
dfgrouped = df.groupby(
    ['campaign_id', 'line_item_id', 'creative_id']
)[['n_clicks', 'n_prints', 'artifact_n_clicks', 'artifact_n_prints']].sum().reset_index()

dfgrouped['bigquery_CTR'] = dfgrouped['n_clicks'] / dfgrouped['n_prints']
dfgrouped['spark_CTR'] = dfgrouped['artifact_n_clicks'] / dfgrouped['artifact_n_prints']

In [ ]:
fig = px.scatter(x='spark_CTR', y='bigquery_CTR', size="log_prints",
                 hover_data={'log_prints': False},
                 custom_data=['n_prints', 'campaign_id', 'line_item_id', 'creative_id'],
                 data_frame=dfgrouped)
fig.update_traces(hovertemplate='Campaign: %{customdata[1]}<br>Line item: %{customdata[2]}<br>Creative: %{customdata[3]}<br>Spark: %{x:.2%} <br>BigQuery: %{y:.2%}<br>Prints: %{customdata[0]}')
fig.write_html('scatter_source_comparison.html', auto_open=False)